# Compute maximum flood extent utilizing STAC

With this notebook, we want to demo how STAC can be used to find GFM
ensemble_flood_extent data and derive the maximum flood extent from it.

In [1]:
# Some necessary imports

from pystac_client import Client
from datetime import datetime
from odc import stac as odc_stac
import pyproj
import rioxarray # noqa
import xarray as xr
from shapely.geometry import box

In [2]:
# Define asset name to use
asset_name = "ensemble_flood_extent"

# Define bounding box
aoi = box(16.77, 49.91, 18.62, 51.25)

# Define time range
time_range = (datetime(2024, 9, 18), datetime(2024, 9, 28))

# EODC STAC API URL
api_url = "https://stac.eodc.eu/api/v1"
eodc_catalog = Client.open(api_url)

# Define search query using pystac_client
search = eodc_catalog.search(
    max_items=1000,
    collections="GFM",
    intersects=aoi,
    datetime=time_range
)

# Get all found items
items = search.item_collection()
print("We found", len(items), "items, that match our filter criteria.")

We found 30 items, that match our filter criteria.


In [3]:
# Derive Equi7Grid CRS from first found item
crs = pyproj.CRS.from_wkt(items[0].properties["proj:wkt2"])

# Load asset data into xarray using odc-stac
# Adjust chunk size of x/y according to available RAM
xx = odc_stac.load(
    items, 
    bbox=aoi.bounds,
    crs=crs,
    bands=["ensemble_flood_extent"],
    dtype="uint8",
    chunks={"x": 5000, "y": 5000, "time": -1}, 
    resolution=20)

Optionally, you can save the xarray as a ZARR data store for future use. It's important to note that the CRS (Coordinate Reference System) needs to be reapplied to the xarray after reading it.

In [4]:
# Optional: Save data as ZARR data store
# xx.to_zarr("./maximum_flood_data/maximum_flood.zarr")

# Read data from ZARR data store 
# xx = xr.open_zarr("./maximum_flood_data/maximum_flood.zarr")
# xx.rio.write_crs(crs, inplace=True)

In [5]:
# Mask out data which is nodata (255) and no-flood (0)
xx = xx.where((xx != 255) & (xx != 0))

# Calculate sum over time dimension
data = xx.sum(dim="time").astype("uint8")

# Save result in current CRS (Equi7Grid)
data.ensemble_flood_extent.rio.to_raster("./maximum_flood_data/result_equi7.tif", compress="LZW")

# Optional: Reproject data to WebMercator (EPSG:3857)
data = data.rio.reproject("EPSG:3857")
data.ensemble_flood_extent.rio.to_raster("./maximum_flood_data/result_epsg3857.tif", compress="LZW")